<a href="https://colab.research.google.com/github/arahrooh31/UCLA_BE223C/blob/Keane_temp/get_dbt_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
import os
import numpy as np
import matplotlib.pyplot as plt

#
# Read Data from google drive
#
from google.colab import drive #for loading gdrive data
from google.colab import files

# install dependencies not included by Colab
# use pip3 to ensure compatibility w/ Google Deep Learning Images 
!pip3 install -q pydicom 
!pip3 install -q tqdm 
!pip3 install -q imgaug 
!pip3 install -q SimpleITK
!pip3 install -q medpy

import SimpleITK as sitk  #for reading mha files
from medpy.io import load #mha files
import pydicom #to read dicom files
from pydicom import dcmread



#
# Load data from google drive
#

drive.mount('/content/gdrive')




Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [67]:
'''
GET DIR LISTING----------------------------------------------------------------
'''
def get_dirs(top_dir): 
    import os

    dir_list = os.scandir(top_dir)
    directories=[]
    patient_data = {}
    for ii in dir_list:
        print('top dir is ',ii)
        if (ii.is_dir()):
            #patient ID directory
            patient_id = ii.name
            patient_data[patient_id]={}
            directories.append(ii.name)
            #get file listings for each directory found
            full_name = os.path.join(top_dir,ii.name)
            print(ii,full_name)

            files = os.scandir(full_name)
            for patient_uid in files:
                print('------ ',patient_uid.name)

            if (patient_uid.is_dir()):
                #this will be the longer directory name
                #with study id
                #This will need to be tokenized to get 
                #the two fields out for info
                #--for now, keep the longer directory name
                patient_data[patient_id][patient_uid.name]={}

                exam_listings = os.scandir(os.path.join(full_name, 
                                                  patient_uid.name))
                for exam_number in exam_listings:
                    patient_data[patient_id][patient_uid.name][exam_number.name] =[]
                    #fill this in with exam dcm contents
                    dcm_list = os.listdir(os.path.join(full_name, 
                                                  patient_uid.name,
                                                  exam_number.name))
                    for dcm_file in dcm_list:
                        patient_data[patient_id][patient_uid.name][exam_number.name].append(dcm_file)

        else:
            print('*** found single file ',ii.name)

    
    return patient_data

In [ ]:
### Main level
top_dir = '/content/gdrive/My Drive/DBT_DATA/TRAINING_DATA/manifest-1605042674814/Breast-Cancer-Screening-DBT'


upper_dir =top_dir

patient_data = get_dirs(upper_dir)


In [ ]:
#
# Show the final DCM files found
#
for patient_id in patient_data.keys():
    for patient_uid in patient_data[patient_id].keys():
        for exam_number in patient_data[patient_id][patient_uid].keys():
            for dcm_file in patient_data[patient_id][patient_uid][exam_number]:
                dcm_full_file = os.path.join(upper_dir,patient_id, patient_uid, exam_number,dcm_file)
                #print(dcm_full_file)
                ds = dcmread(dcm_full_file)
                print(ds.PatientName)
                ds.file_meta.TransferSyntaxUID
                #open a new figure to hold the slice image
                plt.figure()
                plt.imshow(ds.pixel_array[0,:,:], cmap=plt.cm.bone)
                plt.colorbar()
                title_text = str(patient_id) + '_' + str(exam_number)
                plt.title(patient_id)





DBT-P00013


/usr/local/lib/python3.7/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:177: UserWarning: The (0028,0101) 'Bits Stored' value (10-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  f"The (0028,0101) 'Bits Stored' value ({ds.BitsStored}-bit) "


DBT-P00064
DBT-P00064
DBT-P00064
DBT-P00064
DBT-P00052
DBT-P00052
DBT-P00052
DBT-P00052
DBT-P00023
DBT-P00023
DBT-P00023


In [ ]:
 ###
 ### Build links to dcm files
 ###
 print('upper dir = ',upper_dir)
 counter = 0
 patient_folder = {} #store dcm files
 for root, directories, filenames in os.walk(upper_dir): 
    #print('base is ', os.path.basename(directories))
    for directory in directories:
        #create dictionary entry for this directory. The files will be stored
        #within this
        patient_path = os.path.join(root, directory) #link to direct pt folder
        print('root, dir = ',root,patient_path)
        #for every directory found in the patient directory, get a listing of
        #files in each 
        patient_files = os.listdir(patient_path)
        for file in patient_files:
            if ('.dcm' in file):
                if (directory not in patient_folder.keys()):
                    patient_folder[directory] = []
                patient_folder[directory].append(patient_files)
    #for filename in filenames:  
    #    print('f= ',os.path.join(root,filename))
    #    counter = counter + 1
    #    if (counter > 10):
    #        break
    #if (counter > 10):
    #    break

In [ ]:
for ii in patient_folder.keys():
    print(ii)

In [ ]:
### Open dcm file
###
from pydicom import dcmread
file = '/content/gdrive/My Drive/DBT_DATA/TEST_DATA/manifest-1605042674814/Breast-Cancer-Screening-DBT/DBT-P00118/01-01-2000-DBT-S05386-MAMMO screening digital bilateral-70928/7697.000000-17471/1-1.dcm'


ds = dcmread(file)
print(ds.PatientName)
ds.file_meta.TransferSyntaxUID
plt.imshow(ds.pixel_array[0,:,:], cmap=plt.cm.bone) 


In [ ]:
np.shape(ds.pixel_array)
ds_slice = ds.pixel_array[0,:,:]
plt.imshow(ds_slice, cmap=plt.cm.jet)
plt.colorbar()